In [1]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
print(device)

cuda


In [4]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [5]:
train_path='O:\\ninjas\\mamooty-mohanlal\\training_images'
test_path='O:\\ninjas\\mamooty-mohanlal\\testing_images'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [6]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('\\')[-1] for j in root.iterdir()])

In [7]:
print(classes)

['mammooty', 'mohanlal']


In [8]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [9]:
model=ConvNet(num_classes=2).to(device)

In [10]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [16]:
num_epochs=15

In [17]:
train_count=len(glob.glob(train_path+'\\**\\*.jpeg'))
test_count1=len(glob.glob(test_path+'\\**\\*.jpg'))
test_count2=len(glob.glob(test_path+'\\**\\*.jpeg'))
test_count = test_count1+test_count2

In [18]:
print(train_count,test_count)

4761 306


In [19]:
best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy


Epoch: 0 Train Loss: tensor(7.2076) Train Accuracy: 0.5332913253518169 Test Accuracy: 0.49673202614379086
Epoch: 1 Train Loss: tensor(1.6081) Train Accuracy: 0.6305398025624869 Test Accuracy: 0.6568627450980392
Epoch: 2 Train Loss: tensor(1.0782) Train Accuracy: 0.7015332913253518 Test Accuracy: 0.6503267973856209
Epoch: 3 Train Loss: tensor(0.6985) Train Accuracy: 0.7597143457256879 Test Accuracy: 0.6078431372549019
Epoch: 4 Train Loss: tensor(1.0069) Train Accuracy: 0.7454316320100819 Test Accuracy: 0.5522875816993464
Epoch: 5 Train Loss: tensor(0.5640) Train Accuracy: 0.8185255198487713 Test Accuracy: 0.6797385620915033
Epoch: 6 Train Loss: tensor(0.3633) Train Accuracy: 0.8750262549884478 Test Accuracy: 0.7352941176470589
Epoch: 7 Train Loss: tensor(0.2680) Train Accuracy: 0.9038017223272422 Test Accuracy: 0.7189542483660131
Epoch: 8 Train Loss: tensor(0.1660) Train Accuracy: 0.9458097038437303 Test Accuracy: 0.6699346405228758
Epoch: 9 Train Loss: tensor(0.1572) Train Accuracy: 0.